<a href="https://colab.research.google.com/github/sleeprock/SQL/blob/main/product_analytics_training_stepik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задания по SQL. Тренажер по продуктовой аналитике (https://stepik.org/course/83415/syllabus)

#Задача1
Выводит пользователей, которые прослушивают рок, но никогда не слушают поп.

In [ ]:
sql = '''
select distinct t.user_id from listen_table t
where t.music_category = 'rock'
AND t.user_id not in (
select t.user_id from listen_table t
where t.music_category = 'pop'
)
'''

#Задача 2. 
Выводит пользователей, которые оплачивают подписку в первые три дня после регистрации.

In [ ]:
#
sql = '''
with dates as (
select t.user_id, date(t.payment_date) as pay, date(r.registration_date) as reg
from payment_table t
join registration_table r on t.user_id = r.user_id
)
select t.user_id
from dates t
where datediff(t.pay, t.reg) <=2
'''

#
sql = '''
select t.user_id
from payment_table t
join registration_table r on t.user_id = r.user_id
where datediff(t.payment_date, r.registration_date) <=2
'''

#Задача 3.
Первый трек, который прослушал пользователь.

In [ ]:
sql = '''
select t.user_id, t.track_name from listen_table t
where t.listen_date in (
select min(t.listen_date) from listen_table t
group by t.user_id
)
'''

#Задача 4.
Сколько пользователей, которые оплатили подписку, но не слушают музыку?

In [ ]:
#my: join, null
sql = '''
select count(distinct t.user_id) 
from (
select p.user_id,
case when l.user_id is NULL then 1 else 0 end as count
from payment_table p
left join listen_table l on p.user_id = l.user_id
where l.user_id is NULL
  ) t
'''

#not exists
sql = '''
select count(distinct p.user_id) from payment_table p
 where not exists(
 select 1 from listen_table t
 where t.user_id=p.user_id
 )
'''

#Задача 5.
Какой исполнитель самый популярный?

In [ ]:
sql = '''
select t.musician_name, count(t.musician_name) from listen_table t
group by t.musician_name
order by count(t.musician_name) desc limit 1
'''

In [ ]:
services = ['Яндекс', 'Spotify','Apple Podcast','Boom','Last.fm',
'Google Music','Музыка VK','Apple Music']

#2.1
Требуется посчитать: сколько процентов прослушиваний приходится на рекомендации друзей?

Мы собрали данные в таблицу прослушиваний песен пользователями + добавили информацию о лайках + источнике прослушивания и времени прослушивания.

В таблице лежат следующие данные: 

likes - стандартный палец вверх 

likes_1-likes_3 - оценка песни по трем критериям (бит, слова, музыка)

source_id - имеет 4 значения: 

1 - песня из рубрики популярное или подборки приложений, 

2 - песня из сохраненных

3 - песня из рекомендаций 

4 - песня от друзей

song_id user_id like like_1 like_2 like_3 source_id listen_time_percentage

In [ ]:
sql = '''
select 
(
select count(t.source_id) from listen_table t
where t.source_id = 4
) / count(t.source_id) * 100
from listen_table t
'''

#2.2
Теперь мы знаем что 20% прослушиваний приходится на рекомендации от друзей. 

Давайте теперь узнаем — насколько эта музыка нравится пользователям. Посчитаем средний процент времени прослушивания этих песен. 

In [ ]:
sql = '''
select avg(t.listen_time_percentage) from listen_table t
where t.source_id = 4
group by t.source_id
'''

#2.3 
Теперь мы знаем, что 20% прослушиваний приходится на рекомендации от друзей.
В среднем, процент «дослушивания» треков чуть больше 46%, что уже вполне неплохо!

Осталось выяснить, какой процент людей пользуется этой функцией.
Для этого надо посчитать процент тех, кто послушал хотя бы два трека по рекомендации друзей, от всех пользователей. 

In [ ]:
#
sql = '''
select 
(
 select count(*) from (
select t.user_id, count(t.source_id) from listen_table t
where t.source_id = 4
group by t.user_id having count(t.source_id) > 1
) as z
) /
count(distinct t.user_id) * 100
from listen_table t
'''

#with, over
sql = ''' with totcnt as (
select t.user_id, count(t.source_id),
count(*) over() as totalrec
from listen_table t
where t.source_id = 4
group by t.user_id having count(t.source_id) > 1
)
select count(distinct tc.user_id) / count(distinct t.user_id) * 100 from listen_table t
left join totcnt tc on t.user_id = tc.user_id
'''

#2.4
Давайте выясним, как работает наша система рекомендаций музыки (source_id = 3).
Для этого надо посчитать: в скольки процентах случаев пользователь переключал трек рекомендованный системой в первые секунды (возьмем процент «дослушивания» меньше 15%). 

In [ ]:
sql = '''
select (
(
select distinct count(*) from 
(select * from listen_table t where t.source_id = 3 and t.listen_time_percentage <= 15 ) as z
)
/
(
select distinct count(*) from listen_table t
where t.source_id = 3
) * 100
) as prc
'''
